# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
import sys
import os
sys.path.append(os.path.abspath(r"C:\Class\GWU-VIRT-DATA-PT-08-2024-U-LOLC\06-Python-APIs\MOD6_python-api-challenge\The_WeatherPy"))

# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import the API key
from config import Vacation_api_key 

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,isafjordur,66.0755,-23.1240,1.02,64,75,2.57,IS,1728910266
1,1,lorengau,-2.0226,147.2712,27.72,78,49,1.54,PG,1728910268
2,2,lebu,-37.6167,-73.6500,11.43,80,1,4.88,CL,1728910066
3,3,tolanaro,-25.0319,46.9987,26.04,73,20,11.83,MG,1728910270
4,4,bandundu,-3.3167,17.3667,26.69,80,100,1.59,CD,1728910272


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
City_map_plot_1 = city_data_df.hvplot.points("Lng", "Lat",
                                             geo = True,
                                             color = "City",
                                             size = "Humidity",
                                             #frame_width = 600,
                                             #frame_height = 600,
                                             tiles = "OSM")

# Display the map
City_map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Personal weather conditions criteria:
    # The ideal weather condititon Criteria:
    # Maximum temperature must be greater than 21 degrees but lower than 30
    # Hunidity must be less than 75%
    # Cloudiness must be less than 3%
    # Wind speed must be lessthan 5 m/s

# YOUR CODE HERE
weather_conditions_df  = city_data_df.loc[(city_data_df["Max Temp"] < 30)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Humidity"] < 75	)
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 5)]

# Drop any rows with null values
weather_conditions_df.dropna()

# Display sample data
weather_conditions_df 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,cabo san lucas,22.8909,-109.9124,26.53,70,0,3.75,MX,1728910341
236,236,tarija,-21.5355,-64.7296,22.95,68,0,1.54,BO,1728910531
350,350,durres,41.3231,19.4414,24.10,60,0,4.08,AL,1728910660
438,438,kailua-kona,19.6406,-155.9956,23.98,64,0,2.57,US,1728910765
496,496,basse-terre,16.1667,-61.6667,28.58,74,0,3.09,GP,1728910830


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = weather_conditions_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
64,cabo san lucas,MX,22.8909,-109.9124,70,
236,tarija,BO,-21.5355,-64.7296,68,
350,durres,AL,41.3231,19.4414,60,
438,kailua-kona,US,19.6406,-155.9956,64,
496,basse-terre,GP,16.1667,-61.6667,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel

radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":Vacation_api_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
tarija - nearest hotel: Hostal Bolivar
durres - nearest hotel: Hotel Dardania
kailua-kona - nearest hotel: Kona Seaside Hotel
basse-terre - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
64,cabo san lucas,MX,22.8909,-109.9124,70,Comfort Rooms
236,tarija,BO,-21.5355,-64.7296,68,Hostal Bolivar
350,durres,AL,41.3231,19.4414,60,Hotel Dardania
438,kailua-kona,US,19.6406,-155.9956,64,Kona Seaside Hotel
496,basse-terre,GP,16.1667,-61.6667,74,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display
# Configure the map plot

Hotel_map_plot = hotel_df.hvplot.points("Lng", "Lat",
                                             geo = True,
                                             color = "City",
                                             size = "Humidity",
                                             tiles = "OSM",
                                             hover_cols = ["Hotel Name", "Country"])

# Display the map
Hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)